In [1]:
import pandas as pd
import numpy as np
import re
import time

import bs4 as bs4
import json

import glob
import tqdm

pd.set_option("max.columns", 131)

#https://strftime.org/
%matplotlib inline
%pylab inline

Populating the interactive namespace from numpy and matplotlib


In [2]:
df = pd.read_csv("labels_curso - to_label_2.csv", index_col=0).dropna(subset=["y"])

In [3]:
df.duplicated().mean()

0.0

In [4]:
df.duplicated(['watch-title']).mean()

0.0

In [5]:
df.shape

(1164, 16)

In [6]:
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier

In [7]:
df_limpo = pd.DataFrame(index=df.index)
df_limpo['title'] = df['watch-title']

## 1. Limpeza da data

In [8]:
clean_date = df['watch-time-text'].str.extract(r"(\d+) de ([a-z]+)\. de (\d+)")
clean_date[0] = clean_date[0].map(lambda x: "0"+x[0] if len(x) == 1 else x)
#clean_date[1] = clean_date[1].map(lambda x: x[0].upper()+x[1:])

mapa_meses = {"jan": "Jan",
              "fev": "Feb",
              "mar": "Mar", 
              "abr": "Apr", 
              "mai": "May", 
              "jun": "Jun",
              "jul": "Jul",
              "ago": "Aug", 
              "set": "Sep", 
              "out": "Oct", 
              "nov": "Nov",
              "dez": "Dec"}

clean_date[1] = clean_date[1].map(mapa_meses)

clean_date = clean_date.apply(lambda x: " ".join(x), axis=1)
clean_date.head()
df_limpo['date'] = pd.to_datetime(clean_date, format="%d %b %Y")

## 2. Limpeza de Views

In [9]:
views = df['watch-view-count'].str.extract(r"(\d+\.?\d*)", expand=False).str.replace(".", "").fillna(0).astype(int)
df_limpo['views'] = views

## 3. Features

In [10]:
features = pd.DataFrame(index=df_limpo.index)
y = df['y'].copy()

In [11]:
features['tempo_desde_pub'] = (pd.to_datetime("2019-12-03") - df_limpo['date']) / np.timedelta64(1, 'D')
features['views'] = df_limpo['views']
features['views_por_dia'] = features['views'] / features['tempo_desde_pub']
features = features.drop(['tempo_desde_pub'], axis=1)

In [12]:
features.head()

views  views_por_dia
0     28028      61.464912
394    1161      21.109091
393  141646     809.405714
392     325      21.666667
391      61       7.625000

In [13]:
mask_train = df_limpo['date'] < "2019-04-01"
mask_val = (df_limpo['date'] >= "2019-04-01")

Xtrain, Xval = features[mask_train], features[mask_val]
ytrain, yval = y[mask_train], y[mask_val]
Xtrain.shape, Xval.shape, ytrain.shape, yval.shape

((555, 2), (609, 2), (555,), (609,))

In [95]:
from sklearn.feature_extraction.text import TfidfVectorizer

title_train = df_limpo[mask_train]['title']
title_val = df_limpo[mask_val]['title']

title_vec = TfidfVectorizer(min_df=2, ngram_range=(1,4))
title_bow_train = title_vec.fit_transform(title_train)
title_bow_val = title_vec.transform(title_val)


intro to machine learning -> intro, to, machine, learning  -> ngram_range=(1,1)  
intro to machine learning -> intro, to, machine, learning, intro to, to machine, machine learning -> ngram_range=(1,2)  
intro to machine learning -> intro to, to machine, machine learning -> ngram_range=(2,2)  


In [96]:
title_bow_train.shape

(555, 1333)

In [97]:
from scipy.sparse import hstack, vstack

Xtrain_wtitle = hstack([Xtrain, title_bow_train])
Xval_wtitle = hstack([Xval, title_bow_val])

In [98]:
Xtrain_wtitle.shape, Xval_wtitle.shape

((555, 1335), (609, 1335))

# 4 Random Forest

In [85]:
mdl = RandomForestClassifier(n_estimators=1000, random_state=0, min_samples_leaf=1, class_weight="balanced", n_jobs=6)
mdl.fit(Xtrain_wtitle, ytrain)

RandomForestClassifier(bootstrap=True, class_weight='balanced',
            criterion='gini', max_depth=None, max_features='auto',
            max_leaf_nodes=None, min_impurity_decrease=0.0,
            min_impurity_split=None, min_samples_leaf=1,
            min_samples_split=2, min_weight_fraction_leaf=0.0,
            n_estimators=1000, n_jobs=6, oob_score=False, random_state=0,
            verbose=0, warm_start=False)

In [86]:
p = mdl.predict_proba(Xval_wtitle)[:, 1]

In [87]:
from sklearn.metrics import roc_auc_score, average_precision_score

In [88]:
average_precision_score(yval, p)

0.20564709615419985

In [89]:
roc_auc_score(yval, p)

0.6848794008374124

ap 0.1887361518270232, auc 0.6766232234475438
ap 0.1773056089416357, auc 0.66359025771068 - min_samples_leaf=2

ap 0.17595568029276232, auc 0.6454708969747007 - min_df=1  
ap 0.17508239417116303, auc 0.650247685321696 - min_df=3  
ap 0.21140611241061869, auc 0.6785398360559061 - min_df=2, ngram_range=(1,2)  
ap 0.22228951304206077, auc 0.6914990859232175 - min_df=2, ngram_range=(1,3)  
ap 0.20564709615419985, auc 0.6848794008374124 - min_df=2, ngram_range=(1,4)  


ap 0.1834073532858915, auc 0.6716990033614437 - n_estimators=100
ap 0.1762251704396226, auc 0.6700772542312909 - n_estimators=100, min_samples_leaf=2  


RF ap 0.22228951304206077, auc 0.6914990859232175 - min_df=2, ngram_range=(1,3)  

# 5 LightGBM

In [90]:
from lightgbm import LGBMClassifier

/Users/mario/miniconda3/lib/python3.7/site-packages/lightgbm/__init__.py:48: UserWarning: Starting from version 2.2.1, the library file in distribution wheels for macOS is built by the Apple Clang (Xcode_8.3.3) compiler.
This means that in case of installing LightGBM from PyPI via the ``pip install lightgbm`` command, you don't need to install the gcc compiler anymore.
Instead of that, you need to install the OpenMP library, which is required for running LightGBM on the system with the Apple Clang compiler.
You can install the OpenMP library by the following command: ``brew install libomp``.
  "You can install the OpenMP library by the following command: ``brew install libomp``.", UserWarning)


In [99]:
mdl = LGBMClassifier(random_state=0, class_weight="balanced", n_jobs=6)
mdl.fit(Xtrain_wtitle, ytrain)

LGBMClassifier(boosting_type='gbdt', class_weight='balanced',
        colsample_bytree=1.0, importance_type='split', learning_rate=0.1,
        max_depth=-1, min_child_samples=20, min_child_weight=0.001,
        min_split_gain=0.0, n_estimators=100, n_jobs=6, num_leaves=31,
        objective=None, random_state=0, reg_alpha=0.0, reg_lambda=0.0,
        silent=True, subsample=1.0, subsample_for_bin=200000,
        subsample_freq=0)

In [100]:
p = mdl.predict_proba(Xval_wtitle)[:, 1]

/Users/mario/miniconda3/lib/python3.7/site-packages/lightgbm/basic.py:497: UserWarning: Converting data to scipy sparse matrix.
  warnings.warn('Converting data to scipy sparse matrix.')


In [103]:
average_precision_score(yval, p), roc_auc_score(yval, p)

(0.19802541702844073, 0.6238131745002063)

# 6 Bayesian Optimization

In [105]:
from skopt import forest_minimize

In [107]:
def tune_lgbm(params):
    print(params)
    lr = params[0]
    max_depth = params[1]
    min_child_samples = params[2]
    subsample = params[3]
    colsample_bytree = params[4]
    n_estimators = params[5]
    
    min_df = params[6]
    ngram_range = (1, params[7])
    
    title_vec = TfidfVectorizer(min_df=min_df, ngram_range=ngram_range)
    title_bow_train = title_vec.fit_transform(title_train)
    title_bow_val = title_vec.transform(title_val)
    
    Xtrain_wtitle = hstack([Xtrain, title_bow_train])
    Xval_wtitle = hstack([Xval, title_bow_val])
    
    mdl = LGBMClassifier(learning_rate=lr, num_leaves=2 ** max_depth, max_depth=max_depth, 
                         min_child_samples=min_child_samples, subsample=subsample,
                         colsample_bytree=colsample_bytree, bagging_freq=1,n_estimators=n_estimators, random_state=0, 
                         class_weight="balanced", n_jobs=6)
    mdl.fit(Xtrain_wtitle, ytrain)
    
    p = mdl.predict_proba(Xval_wtitle)[:, 1]
    
    print(roc_auc_score(yval, p))
    
    return -average_precision_score(yval, p)


space = [(1e-3, 1e-1, 'log-uniform'), # lr
          (1, 10), # max_depth
          (1, 20), # min_child_samples
          (0.05, 1.), # subsample
          (0.05, 1.), # colsample_bytree
          (100,1000), # n_estimators
          (1,5), # min_df
          (1,5)] # ngram_range

res = forest_minimize(tune_lgbm, space, random_state=160745, n_random_starts=20, n_calls=50, verbose=1)

Iteration No: 1 started. Evaluating function at random point.
[0.009944912110647982, 5, 1, 0.4677107511929402, 0.49263223036174764, 272, 3, 1]


/Users/mario/miniconda3/lib/python3.7/site-packages/lightgbm/basic.py:497: UserWarning: Converting data to scipy sparse matrix.
  warnings.warn('Converting data to scipy sparse matrix.')


0.6016099545910243
Iteration No: 1 ended. Evaluation done at random point.
Time taken: 0.2389
Function value obtained: -0.1404
Current minimum: -0.1404
Iteration No: 2 started. Evaluating function at random point.
[0.053887464791860025, 1, 15, 0.7437489153990157, 0.8675167974293533, 549, 3, 4]


/Users/mario/miniconda3/lib/python3.7/site-packages/lightgbm/basic.py:497: UserWarning: Converting data to scipy sparse matrix.
  warnings.warn('Converting data to scipy sparse matrix.')


0.5507312614259597
Iteration No: 2 ended. Evaluation done at random point.
Time taken: 0.2130
Function value obtained: -0.1203
Current minimum: -0.1404
Iteration No: 3 started. Evaluating function at random point.
[0.004151454520895999, 6, 20, 0.8682075103820793, 0.9491436163200662, 411, 4, 3]


...

0.6480214660612137
Iteration No: 48 ended. Search finished for the next optimal point.
Time taken: 2.0176
Function value obtained: -0.1582
Current minimum: -0.2378
Iteration No: 49 started. Searching for the next optimal point.
[0.06892466655245144, 7, 5, 0.8261588403741187, 0.07729616979076112, 853, 2, 5]


/Users/mario/miniconda3/lib/python3.7/site-packages/lightgbm/basic.py:497: UserWarning: Converting data to scipy sparse matrix.
  warnings.warn('Converting data to scipy sparse matrix.')


0.6603467594503744
Iteration No: 49 ended. Search finished for the next optimal point.
Time taken: 1.8144
Function value obtained: -0.1719
Current minimum: -0.2378
Iteration No: 50 started. Searching for the next optimal point.
[0.06602730762014003, 10, 2, 0.6724229168392452, 0.07445872598097698, 855, 2, 5]


/Users/mario/miniconda3/lib/python3.7/site-packages/lightgbm/basic.py:497: UserWarning: Converting data to scipy sparse matrix.
  warnings.warn('Converting data to scipy sparse matrix.')


0.6782449725776964
Iteration No: 50 ended. Search finished for the next optimal point.
Time taken: 3.1390
Function value obtained: -0.2132
Current minimum: -0.2378


ap 0.22228951304206077, auc 0.6914990859232175 - min_df=2, ngram_range=(1,3) RF 
ap 0.2378, auc 0.6883293035324645 LGBM - [0.08265121231498246, 7, 1, 0.7251351011494334, 0.07547006552546137, 839, 2, 5]


In [108]:
res.x

[0.08265121231498246, 7, 1, 0.7251351011494334, 0.07547006552546137, 839, 2, 5]

# 7 Logistic Reg

In [109]:
from sklearn.preprocessing import MaxAbsScaler, StandardScaler
from scipy.sparse import csr_matrix

In [115]:
Xtrain_wtitle2 = csr_matrix(Xtrain_wtitle.copy())
Xval_wtitle2 = csr_matrix(Xval_wtitle.copy())

#scaler = StandardScaler()
scaler = MaxAbsScaler()


#Xtrain_wtitle2[:, :2] = scaler.fit_transform(Xtrain_wtitle2[:, :2].todense())
#Xval_wtitle2[:, :2] = scaler.transform(Xval_wtitle2[:, :2].todense())

Xtrain_wtitle2 = scaler.fit_transform(Xtrain_wtitle2)
Xval_wtitle2 = scaler.transform(Xval_wtitle2)

In [116]:
Xval_wtitle2.shape

(609, 1335)

In [135]:

mdl = LogisticRegression(C=0.5,n_jobs=6, random_state=0)
mdl.fit(Xtrain_wtitle2, ytrain)

/Users/mario/miniconda3/lib/python3.7/site-packages/sklearn/linear_model/logistic.py:433: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)
/Users/mario/miniconda3/lib/python3.7/site-packages/sklearn/linear_model/logistic.py:1300: UserWarning: 'n_jobs' > 1 does not have any effect when 'solver' is set to 'liblinear'. Got 'n_jobs' = 6.
  " = {}.".format(effective_n_jobs(self.n_jobs)))


LogisticRegression(C=0.5, class_weight=None, dual=False, fit_intercept=True,
          intercept_scaling=1, max_iter=100, multi_class='warn', n_jobs=6,
          penalty='l2', random_state=0, solver='warn', tol=0.0001,
          verbose=0, warm_start=False)

In [136]:
p = mdl.predict_proba(Xval_wtitle2)[:, 1]

In [137]:
average_precision_score(yval, p), roc_auc_score(yval, p)

(0.21340786207179874, 0.6835525151854692)

In [ ]:
(0.20616279322296893, 0.6606416229285841) - sem tuning, standardscaler

(0.20757989629841797, 0.6862357728371764) - sem tuning, maxabsscaler
(0.18953863996413214, 0.6741463702305833) - C=10, maxabsscaler

(0.21340786207179874, 0.6835525151854692) - C=0.5, maxabsscaler